In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def map_ranges(inputValue: float, inMin: float, inMax: float, outMin: float, outMax: float):
    """
    Map a given value from range 1 -> range 2
    :param inputValue: The value you want to map
    :param inMin: Minimum Value of Range 1
    :param inMax: Maximum Value of Range 1
    :param outMin: Minimum Value of Range 2
    :param outMax: Maximum Value of Range 2
    :return: The new Value in Range 2
    """
    slope = (outMax - outMin) / (inMax - inMin)
    return outMin + slope * (inputValue - inMin)

def histogram(data: np.array, bins_num: int = 255):
    """

    :param data:
    :param bins_num:
    :return:
    """
    if bins_num == 2:
        new_data = data
    else:
        new_data = np.round(np.interp(data, (data.min(), data.max()), (0, bins_num))).astype('uint8')
    bins = np.arange(0, bins_num)
    hist = np.bincount(new_data.ravel(), minlength=bins_num)
    return hist, bins

In [ ]:
image1_bgr = cv2.imread('src/Images/Lenna_512.png')
image1_rgb = cv2.cvtColor(image1_bgr, cv2.COLOR_BGR2RGB)
image1_gray = cv2.cvtColor(image1_bgr, cv2.COLOR_BGR2GRAY)
image1_size = image1_rgb.shape[0] * image1_rgb.shape[1]

image2_bgr = cv2.imread('src/Images/hand_256.png')
image2_gray = cv2.cvtColor(image2_bgr, cv2.COLOR_BGR2GRAY)

norm_img = np.zeros((image1_rgb.shape[0], image1_rgb.shape[1]))
final_img = cv2.normalize(image1_rgb,  norm_img, 0, 255, cv2.NORM_MINMAX)

# show original image
plt.subplot(121)
plt.imshow(image1_rgb)
plt.title('Original Image')
plt.set_cmap('gray')

# show Normalized image
plt.subplot(122)
plt.imshow(final_img)
plt.title('Normalized Image')
plt.set_cmap('gray')
plt.show()

In [ ]:
hist, bins = histogram(data=image1_rgb, bins_num=255)
plt.hist(image1_rgb.ravel(), bins=255, fc='k', ec='k')
plt.show()

In [ ]:
image1_r = image1_rgb[:, :, 0]
image1_g = image1_rgb[:, :, 1]
image1_b = image1_rgb[:, :, 2]

In [ ]:
# Compute the discrete PDF (Probability distribution function)
pr = hist / image1_size

# Compute the discrete CDF (Accumulative Histogram)
acc = []
temp = 0
for i in range(256):
    temp += pr[i]
    acc.append(temp)
    
acc = np.array(acc) * (image1_rgb.shape[0]-1)
sk = np.round(acc)
# print(sk)

In [ ]:
# map the pixels values
eq_img = np.zeros((256, 256))
for i in range(256):
    for j in range(256):
        eq_img[i][j] = sk[j]



In [ ]:
equ = cv2.equalizeHist(image1_gray)

# show original Image
plt.subplot(121)
plt.imshow(image1_gray)
plt.title('Original Image')
plt.set_cmap('gray')

# show Equalized Image
plt.subplot(122)
plt.imshow(equ)
plt.title('Equalized Image')
plt.set_cmap('gray')
plt.show()

# show de7k images
plt.subplot(111)
plt.imshow(eq_img)
plt.title('de7k Image')
plt.set_cmap('gray')
plt.show()

In [ ]:
plt.hist(equ.ravel(), bins=16, range=[0, 256], fc='k', ec='k')
plt.show()

plt.hist(eq_img.ravel(), bins=16, range=[0, 256], fc='k', ec='k')
plt.show()

In [ ]:
def imhist(im):
  # calculates normalized histogram of an image
	m, n = im.shape
	h = [0.0] * 256
	for i in range(m):
		for j in range(n):
			h[im[i, j]]+=1
	return np.array(h)/(m*n)

def cumsum(h):
	# finds cumulative sum of a numpy array, list
	return [sum(h[:i+1]) for i in range(len(h))]

def histeq(im):
	#calculate Histogram
	h = imhist(im)
	cdf = np.array(cumsum(h)) #cumulative distribution function
	sk = np.uint8(255 * cdf) #finding transfer function values
	s1, s2 = im.shape
	Y = np.zeros_like(im)
	# applying transfered values for each pixels
	for i in range(0, s1):
		for j in range(0, s2):
			Y[i, j] = sk[im[i, j]]
	H = imhist(Y)
	#return transformed image, original and new istogram, 
	# and transform function
	return Y , h, H, sk

In [ ]:
import pylab as plt
import matplotlib.image as mpimg
import numpy as np
# load image to numpy arrayb
# matplotlib 1.3.1 only supports png images
# use scipy or PIL for other formats
img = np.uint8(mpimg.imread('src/Images/cat256.jpg'))
# convert to grayscale
# do for individual channels R, G, B, A for nongrayscale images

img = np.uint8((0.2126* img[:,:,0]) + \
  		np.uint8(0.7152 * img[:,:,1]) +\
			 np.uint8(0.0722 * img[:,:,2]))

# use hist module from hist.py to perform histogram equalization
new_img, h, new_h, sk = histeq(img)

# show old and new image
# show original image
plt.subplot(121)
plt.imshow(img)
plt.title('original image')
plt.set_cmap('gray')
# show original image
plt.subplot(122)
plt.imshow(new_img)
plt.title('hist. equalized image')
plt.set_cmap('gray')
plt.show()

# plot histograms and transfer function
fig = plt.figure()
fig.add_subplot(221)
plt.plot(h)
plt.title('Original histogram') # original histogram

fig.add_subplot(222)
plt.plot(new_h)
plt.title('New histogram') #hist of eqlauized image

fig.add_subplot(223)
plt.plot(sk)
plt.title('Transfer function') #transfer function

plt.show()

In [ ]:
def threshold_image(data: np.ndarray, threshold: int, type: str = "global"):
    if type == "global":
        return (data > threshold).astype(int)
    
    elif type == "local":
        pass


def normalize_image(data: np.ndarray, value: int):
    data = data / value
    print('Min: %.3f, Max: %.3f' % (data.min(), data.max()))
    return data

In [ ]:
out_threshold = threshold_image(img, 130)
out_threshold

In [ ]:
imgplot = plt.imshow(out_threshold, cmap="gray")

In [ ]:
plt.hist(out_threshold.ravel())
plt.show()

In [ ]:
print(img.dtype)
out_norm = normalize_image(img, 220)
out_norm
# new_data = np.round(np.interp(out_norm, (out_norm.min(), out_norm.max()), (0, 255))).astype('uint8')

In [ ]:
plt.imshow(img, cmap="gray")

In [ ]:
print(out_norm)
plt.imshow(out_norm, cmap="gray")

In [ ]:
ar = np.array(img).astype(np.float32)
mn = np.min(ar)
mx = np.max(ar)
norm = (ar - mn) * (1.0 / (mx - mn))
norm

In [ ]:
plt.imshow(norm, cmap="gray")

In [ ]:
# techniques on the input image
thresh1 = cv2.adaptiveThreshold(image1_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                          cv2.THRESH_BINARY, 199, 5)
  
thresh2 = cv2.adaptiveThreshold(image1_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY, 199, 5)

# show original image
plt.subplot(121)
plt.imshow(thresh1)
plt.title('thresh1')
plt.set_cmap('gray')

# show original image
plt.subplot(122)
plt.imshow(thresh2)
plt.title('thresh2')
plt.set_cmap('gray')
plt.show()

# RGB TO GRAY

In [ ]:
def rgb_to_gray(data: np.ndarray):
#     gray_img = np.zeros((data.shape[0], data.shape[1]))
    
    return np.dot(data[..., :3], [0.299, 0.587, 0.114])
        
#     for i in range(data.shape[0]):
#         gray_img[i] = np.round((data[i][0][0] + data[i][0][0] + data[i][0][0]) / 3)
        
#     return gray_img

In [ ]:
img_bgr = cv2.imread('src/Images/Lenna_512.png')
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

gray = rgb_to_gray(img_rgb)
gray

In [ ]:
# show original image
plt.subplot(121)
plt.imshow(img_rgb)
plt.title('RGB')
# plt.set_cmap('gray')

# show original image
plt.subplot(122)
plt.imshow(gray)
plt.title('GRAY')
# plt.set_cmap('gray')
plt.show()

In [ ]:
block_image = image1_gray[:256, :256]
print(block_image)
print(block_image.shape)

# show blocked image
plt.subplot(121)
plt.imshow(block_image)
plt.title('block_image')
plt.set_cmap('gray')
plt.show()

In [ ]:
size = 256
blocks_num = (image1_gray.shape[0] // size) * 2
print(f"Blocks: {blocks_num}")

all_images = []

for i in range(blocks_num // 2):
    for j in range(blocks_num // 2):
        print(f"Block From {i*size} to {(j+1)*size}")
        
        block_image = image1_gray[j*size:(j+1)*size, i*size:(i+1)*size]
        # 0 [0:256   , 0:256]
        # 1 [256:512 , 0:256]
        
        # 2 []
        # 3 []
        
        all_images.append(block_image)
        
# for i in range(0, image1_gray.shape[0], size):
#     for j in range(0, image1_gray.shape[1], size):
#         print(f"Block From {i} to {(size)*k}")
#         block_image = image1_gray[j:size*(k), j:size*(k)]
#         all_images.append(block_image)
#         k += 1
    
print(len(all_images))

# for i in range(0, blocks_num):
#     plt.imshow(all_images[i])
#     plt.title('block_image')
#     plt.set_cmap('gray')
#     plt.show()